In [1]:
# On terminal: conda activate python38

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import ipynb.fs.defs.functions as fct
import pickle

import warnings
warnings.filterwarnings("error")

In [3]:
# Load IDs
with open('uniqueIDs.pkl', 'rb') as f:
    uniqueIDs = pickle.load(f)

# Fit mod10 for each participant

In [4]:
# Model settings

# Functions
value_fct = fct.rescorla_wagner
dec_fct = fct.my_softmax_shrinking_2_press_bias

# Store everything
mod_info = {}
mod_info['name'] = 'model10'
mod_info['value_fct'] = value_fct.__name__
mod_info['dec_fct'] = dec_fct.__name__
mod_info['param_names'] = ['v0', 'alpha', 'beta', 'pi', 'pi_t']
#print(mod['value_fct'].__name__)

# save
all_users_folder = 'data/all_users/mod10/'
file_name = all_users_folder+'mod_parameters.pkl'
with open(file_name, 'wb') as f:
    pickle.dump(mod_info, f)

In [5]:
run_ = True

# Folder
all_users_folder = 'data/all_users/mod10/'
    
if run_:
    
    # Parameter range for initial guess 
    # order of mod['param_names']:
    # ['v0', 'alpha', 'beta', 'pi', 'pi_t']
    param_lower_bound = [-5, 0, 0, -10, -10]
    param_upper_bound = [ 5, 1, 15, 10, 10]

    # Fit
    all_users = {}
    p_hit_per_trial = pd.DataFrame([])
    ev_per_trial = pd.DataFrame([])
    for n_part,ID in enumerate(uniqueIDs): 

        # Get data
        user_folder = 'data/user_' + ID + '/'
        df2_cf = pd.read_pickle(user_folder + 'df2_cf.pkl')
        isHit_all_cues, fbs_all_cues, trialNo_all_cues = fct.extract_hits_fbs(df2_cf)

        # Create a new Model object
        mod = fct.Model(mod_name = mod_info['name'],
                     value_fct = value_fct, 
                     dec_fct = dec_fct, 
                     param_names = mod_info['param_names'])

        # Input data to model
        mod.set_data(ID, fbs_all_cues, isHit_all_cues, trialNo_all_cues)

        # Fit model
        mod.fit(param_lower_bound, param_upper_bound, n_iterations=5)

        # Nested dictionnary user data
        all_users[n_part] = {}
        all_users[n_part]['ID']=mod.ID
        all_users[n_part]['nLL']=mod.nLL
        all_users[n_part]['Ntrials']=mod.Ntrials
        all_users[n_part]['Nparams']=len(mod.param_names)
        for i in range(0,len(mod.param_names)):
            all_users[n_part][mod.param_names[i]]=mod.param_values[i]
        
        # Concatenated model predictions: p hit
        tmp = pd.DataFrame(mod.p_hit).transpose()
        tmp.columns = tmp.columns+1
        tmp = tmp.reset_index().rename(columns={'index': 'Cue'})
        tmp.insert(0,'ID',ID)
        p_hit_per_trial = pd.concat([tmp, p_hit_per_trial], axis=0)
        
        # Concatenated model predictions: EVs
        tmp = pd.DataFrame(mod.v).transpose()
        tmp.columns = tmp.columns+1
        tmp = tmp.reset_index().rename(columns={'index': 'Cue'})
        tmp.insert(0,'ID',ID)
        ev_per_trial = pd.concat([tmp, ev_per_trial], axis=0)
    
    # Save mod LLs and parameter values
    mod_fit = pd.DataFrame(all_users).transpose()
    mod_fit.to_pickle(all_users_folder+'mod_param_fits.pkl')
    
    # Save mod predictions
    p_hit_per_trial = p_hit_per_trial.sort_values(by='ID').reset_index(drop=True)
    p_hit_per_trial.to_pickle(all_users_folder+'mod_p_hit_per_trial.pkl')
    ev_per_trial = ev_per_trial.sort_values(by='ID').reset_index(drop=True)
    ev_per_trial.to_pickle(all_users_folder+'mod_ev_per_trial.pkl')
    

In [6]:
mod_fit

,ID,nLL,Ntrials,Nparams,v0,alpha,beta,pi,pi_t
0,001,49.534234,112,5,1.056486,0.009533,8.01498,-1.170754,0.0693
1,003,49.151288,112,5,-0.547195,0.00582,12.636626,0.441126,0.191795
2,006,53.088243,112,5,-0.772136,0.172577,1.068012,-0.206795,1.358687
3,007,49.264696,112,5,0.362211,0.011852,5.862722,-0.314848,0.249557
4,008,37.695428,112,5,-1.679586,0.172165,1.845717,0.218024,1.552458
...,...,...,...,...,...,...,...,...,...
165,453,36.702522,112,5,-1.012415,0.230749,1.009884,-1.290486,6.187478
166,462,35.689832,112,5,-0.9528,0.008323,15.0,0.685107,0.36654
167,469,3.433485,112,5,-0.279127,0.049097,15.0,-0.250036,0.672397
168,470,36.750506,112,5,-1.554709,0.012542,10.664294,1.095368,0.644432


In [13]:
beta = 6
pi_t = 10

vt_all_cues = mod.v
trialNo_all_cues = mod.trialNo_all_cues

# Iterate over cues
for cue, vt in vt_all_cues.items():

    trials = trialNo_all_cues[cue]

    x = beta * (vt + pi_t)

In [14]:
cue

'LP'

In [15]:
# Iterate over cues
for cue, vt in vt_all_cues.items():
    
    # Reinitialise each trial of run 2 (above 56)
    trials_per_run_lst = []
    run_size = 56
    for trial in trials:
        if trial > run_size:
            trials_per_run_lst.append(trial-run_size)
        else:
            trials_per_run_lst.append(trial)

    # Convert to array
    trials_per_run = np.array(trials_per_run_lst)

    # Higher press bias with small trial numbers
    x = beta * (vt + pi_t/trials_per_run)

In [24]:
1/trials_per_run

array([0.5       , 0.33333333, 0.16666667, 0.1       , 0.09090909,
       0.08333333, 0.07142857, 0.06666667, 0.04166667, 0.03846154,
       0.03703704, 0.02857143, 0.02777778, 0.01851852, 0.14285714,
       0.11111111, 0.07142857, 0.05555556, 0.05263158, 0.04166667,
       0.03846154, 0.03571429, 0.03125   , 0.02777778, 0.02325581,
       0.02      , 0.01923077, 0.01851852])

In [25]:
Ntrials = 56
(Ntrials - trials_per_run)/Ntrials

array([0.96428571, 0.94642857, 0.89285714, 0.82142857, 0.80357143,
       0.78571429, 0.75      , 0.73214286, 0.57142857, 0.53571429,
       0.51785714, 0.375     , 0.35714286, 0.03571429, 0.875     ,
       0.83928571, 0.75      , 0.67857143, 0.66071429, 0.57142857,
       0.53571429, 0.5       , 0.42857143, 0.35714286, 0.23214286,
       0.10714286, 0.07142857, 0.03571429])

In [16]:
vt

array([-1.01726584, -1.12041801, -1.22089855, -1.21517731, -1.20960425,
       -1.20417554, -1.19888742, -1.19373627, -1.03331951, -1.13605589,
       -1.08073239, -1.18224078, -1.02212176, -1.12514815, -1.07010716,
       -0.91289238, -1.0187478 , -1.01826223, -1.01778925, -1.12092785,
       -0.96239682, -0.91157107, -0.91386136, -1.01969168, -1.01918167,
       -0.9668852 , -1.07134221, -1.17309381])

In [17]:
shrink = (Ntrials - trials_per_run)/Ntrials

array([ 3.98273416,  2.21291533,  0.44576812, -0.21517731, -0.30051334,
       -0.3708422 , -0.48460171, -0.52706961, -0.61665285, -0.75144051,
       -0.71036202, -0.8965265 , -0.74434398, -0.93996297,  0.35846427,
        0.19821873, -0.30446209, -0.46270668, -0.49147346, -0.70426119,
       -0.57778144, -0.55442821, -0.60136136, -0.74191391, -0.78662353,
       -0.7668852 , -0.87903452, -0.98790862])

In [ ]:
vt + pi_t * shrink